you should first git concerto with
git clone git@github.com:melobio/Concerto-reproducibility.git
add an path to it

In [1]:
import os
import sys
import numpy as np
import scanpy as sc
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


sys.path.append("Concerto-reproducibility")
from concerto_function5_3 import *
from os.path import join as pj

os.environ['CUDA_VISIBLE_DEVICES']='1'

2023-05-13 09:23:45.256364: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
#Select an available GPU to run on a multi-GPU computer or you can run it directly on the CPU without executing this cell
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2023-05-13 09:23:47.648613: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-05-13 09:23:47.753429: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:40:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.74GiB deviceMemoryBandwidth: 1.03TiB/s
2023-05-13 09:23:47.753507: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-05-13 09:23:47.770733: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-05-13 09:23:47.770933: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2023-05-13 09:23:47.773729: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.

In [3]:
task = 'wnn'
experiment = 'concerto_ref_query_mapping'
result_path = pj('../../../result', task, experiment)
data_root = '/opt/data/private/xx/code/MIRACLE/data/raw/rna+adt'
save_path = '../concerto/online_cellmask/'

### read data (h5ad)

In [4]:
adata_RNA_all = sc.read_h5ad(pj(data_root, task,'rna_cellmask.h5ad'))
adata_Protein_all = sc.read_h5ad(pj(data_root, task,'adt_cellmask.h5ad'))

In [5]:
weight_path = save_path + 'weight/'

In [6]:
for i in range(7):
    print('step ', i)
    print('load batch %d as reference')
    adata_RNA = adata_RNA_all[adata_RNA_all.obs['batch']==i]
    adata_Protein = adata_Protein_all[adata_Protein_all.obs['batch']==i]

    if i==0:
        adata_RNA = preprocessing_rna(adata_RNA,min_features = 0, is_hvg=False,batch_key='batch')
        adata_Protein = preprocessing_rna(adata_Protein,min_features = 0, is_hvg=False,batch_key='batch')

    print('load batch %d as reference' % (i+1))
    
    adata_RNA_query = adata_RNA_all[adata_RNA_all.obs['batch']==(i+1)]
    adata_Protein_query = adata_Protein_all[adata_Protein_all.obs['batch']==(i+1)]
    
    adata_RNA_query = preprocessing_rna(adata_RNA_query,min_features = 0, is_hvg=False,batch_key='batch')
    
    adata_Protein_query = preprocessing_rna(adata_Protein_query,min_features = 0, is_hvg=False,batch_key='batch')
    RNA_tf_path = concerto_make_tfrecord(adata_RNA,tf_path = save_path + 'tfrecord/RNA_tf/',batch_col_name = 'batch')
    Protein_tf_path = concerto_make_tfrecord(adata_Protein,tf_path = save_path + 'tfrecord/Protein_tf/',batch_col_name = 'batch')
    RNA_tf_path_query = concerto_make_tfrecord(adata_RNA_query,tf_path = save_path + 'tfrecord/RNA_tf_query/',batch_col_name = 'batch')
    Protein_tf_path_query = concerto_make_tfrecord(adata_Protein_query,tf_path = save_path + 'tfrecord/Protein_tf_query/',batch_col_name = 'batch')
    

    super_parameters = {
        'batch_size': 64, 
        'epoch_pretrain': 3 if i == 0 else 0,
        'epoch_fineturn': 1, 
        'lr': 1e-4,
        'drop_rate': 0.1
        }
    concerto_train_ref_query_multimodality(RNA_tf_path, Protein_tf_path, RNA_tf_path_query, Protein_tf_path_query, weight_path, super_parameters=super_parameters)
    # save_path = './'
    weight_path = save_path + 'weight/'
    rna_tf_path = save_path +'./tfrecord/RNA_tf'                                                                                                                                                                             
    protein_tf_path = save_path +'./tfrecord/Protein_tf'
    rna_tf_path_query = save_path +'./tfrecord/RNA_tf_query'
    protein_tf_path_query = save_path +'./tfrecord/Protein_tf_query'

    source_data_feature, source_data_batch, RNA_id_all, attention_weight = concerto_test_multimodal(weight_path, rna_tf_path, protein_tf_path)
    query_data_feature, query_data_batch, query_RNA_id_all, attention_weight = concerto_test_multimodal(weight_path, rna_tf_path_query, protein_tf_path_query)
    # ref_embedding,query_embedding,ref_id,query_id = concerto_test_multimodal(weight_path,ref_tf_path,query_tf_path,super_parameters=None)
    adata_RNA = adata_RNA[RNA_id_all]
    adata_RNA_query = adata_RNA_query[query_RNA_id_all]
    adata_RNA.obsm['X_embedding'] = source_data_feature
    adata_RNA_query.obsm['X_embedding'] = query_data_feature
    adata_all = adata_RNA.concatenate(adata_RNA_query)

step  0
load batch %d as reference
Processed dataset shape: (6142, 3617)
Processed dataset shape: (6142, 224)
load batch 1 as reference


/root/anaconda3/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number
/root/anaconda3/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


Processed dataset shape: (3978, 3617)
Processed dataset shape: (3978, 224)
AnnData object with n_obs × n_vars = 6142 × 3617
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch', 'n_genes'
    uns: 'log1p'


2023-05-13 09:23:49.582016: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-13 09:23:49.593950: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:40:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.74GiB deviceMemoryBandwidth: 1.03TiB/s
2023-05-13 09:23:49.596286: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2023-05-13 09:23:49.596684: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-05-13 09:23:50.772365: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnec

counter: 1000 shape: (3617,), batch: 0
counter: 2000 shape: (3617,), batch: 0
counter: 3000 shape: (3617,), batch: 0
counter: 4000 shape: (3617,), batch: 0
counter: 5000 shape: (3617,), batch: 0
counter: 6000 shape: (3617,), batch: 0
AnnData object with n_obs × n_vars = 6142 × 224
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch', 'n_genes'
    uns: 'log1p'
counter: 1000 shape: (224,), batch: 0
counter: 2000 shape: (224,), batch: 0
counter: 3000 shape: (224,), batch: 0
counter: 4000 shape: (224,), batch: 0
counter: 5000 shape: (224,), batch: 0
counter: 6000 shape: (224,), batch: 0
AnnData object with n_obs × n_vars = 3978 × 3617
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch', 'n_genes'
    uns: 'log1p'
counter: 1000 shape: (3617,), batch: 0
counter: 2000 shape: (3617,), batch: 0
counter: 3000 shape: (3617,), batch: 0
AnnData object with n_obs × n_vars = 3978 × 224
    obs: 'orig.ident', 'nC

...

/root/anaconda3/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/root/anaconda3/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


Processed dataset shape: (8770, 3617)
Processed dataset shape: (8770, 224)
View of AnnData object with n_obs × n_vars = 5265 × 3617
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch'
counter: 1000 shape: (3617,), batch: 0
counter: 2000 shape: (3617,), batch: 0
counter: 3000 shape: (3617,), batch: 0
counter: 4000 shape: (3617,), batch: 0
counter: 5000 shape: (3617,), batch: 0
View of AnnData object with n_obs × n_vars = 5265 × 224
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch'
counter: 1000 shape: (224,), batch: 0
counter: 2000 shape: (224,), batch: 0
counter: 3000 shape: (224,), batch: 0
counter: 4000 shape: (224,), batch: 0
counter: 5000 shape: (224,), batch: 0
AnnData object with n_obs × n_vars = 8770 × 3617
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch', 'n_genes'
    uns: 'log1p'
counter: 1000 shape: (3617,), batch: 0
counter: 2000 shape:

/root/anaconda3/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],
/root/anaconda3/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


step  6
load batch %d as reference
load batch 7 as reference
Processed dataset shape: (8578, 3617)
Processed dataset shape: (8578, 224)
View of AnnData object with n_obs × n_vars = 8770 × 3617
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch'
counter: 1000 shape: (3617,), batch: 0
counter: 2000 shape: (3617,), batch: 0
counter: 3000 shape: (3617,), batch: 0
counter: 4000 shape: (3617,), batch: 0
counter: 5000 shape: (3617,), batch: 0
counter: 6000 shape: (3617,), batch: 0
counter: 7000 shape: (3617,), batch: 0
counter: 8000 shape: (3617,), batch: 0
View of AnnData object with n_obs × n_vars = 8770 × 224
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch'
counter: 1000 shape: (224,), batch: 0
counter: 2000 shape: (224,), batch: 0
counter: 3000 shape: (224,), batch: 0
counter: 4000 shape: (224,), batch: 0
counter: 5000 shape: (224,), batch: 0
counter: 6000 shape: (224,), batch: 0
counter: 7000 sha

/root/anaconda3/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


### load trained model and predict

In [7]:
emb = [None for i in range(8)]
data_batch = [None for i in range(8)]
RNA_id_all = [None for i in range(8)]
attention_weight = [None for i in range(8)]
adata = []
for i in range(8):
    print(i)
    adata_RNA = adata_RNA_all[adata_RNA_all.obs['batch']==i]
    adata_Protein = adata_Protein_all[adata_Protein_all.obs['batch']==i]
    adata_RNA = adata_RNA
    adata_RNA = preprocessing_rna(adata_RNA,min_features = 0, is_hvg=False,batch_key='batch')
    adata_Protein = preprocessing_rna(adata_Protein,min_features = 0, is_hvg=False,batch_key='batch')

    RNA_tf_path = concerto_make_tfrecord(adata_RNA,tf_path = save_path + 'tfrecord/RNA_tf_batch_%d/'%(i+1),batch_col_name = 'batch')
    Protein_tf_path = concerto_make_tfrecord(adata_Protein,tf_path = save_path + 'tfrecord/Protein_tf_batch_%d/'%(i+1),batch_col_name = 'batch')
    emb[i], data_batch[i], RNA_id_all[i], attention_weight[i] = concerto_test_multimodal(weight_path, RNA_tf_path, Protein_tf_path)
    adata_RNA = adata_RNA[RNA_id_all[i]]
    adata_RNA.obsm['X_embedding'] = emb[i]
    adata.append(adata_RNA)

0
Processed dataset shape: (6142, 3617)
Processed dataset shape: (6142, 224)
AnnData object with n_obs × n_vars = 6142 × 3617
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch', 'n_genes'
    uns: 'log1p'


/root/anaconda3/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


counter: 1000 shape: (3617,), batch: 0
counter: 2000 shape: (3617,), batch: 0
counter: 3000 shape: (3617,), batch: 0
counter: 4000 shape: (3617,), batch: 0
counter: 5000 shape: (3617,), batch: 0
counter: 6000 shape: (3617,), batch: 0
AnnData object with n_obs × n_vars = 6142 × 224
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch', 'n_genes'
    uns: 'log1p'
counter: 1000 shape: (224,), batch: 0
counter: 2000 shape: (224,), batch: 0
counter: 3000 shape: (224,), batch: 0
counter: 4000 shape: (224,), batch: 0
counter: 5000 shape: (224,), batch: 0
counter: 6000 shape: (224,), batch: 0
../concerto/online_cellmask/tfrecord/RNA_tf_batch_1/tf_0.tfrecord
../concerto/online_cellmask/tfrecord/Protein_tf_batch_1/tf_0.tfrecord
all_samples num:32
all_samples num:64
all_samples num:96
all_samples num:128
all_samples num:160
all_samples num:192
all_samples num:224
all_samples num:256
all_samples num:288
all_samples num:320
all_samples num:352
all_samples 

/root/anaconda3/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


counter: 1000 shape: (3617,), batch: 0
counter: 2000 shape: (3617,), batch: 0
counter: 3000 shape: (3617,), batch: 0
AnnData object with n_obs × n_vars = 3978 × 224
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch', 'n_genes'
    uns: 'log1p'
counter: 1000 shape: (224,), batch: 0
counter: 2000 shape: (224,), batch: 0
counter: 3000 shape: (224,), batch: 0
../concerto/online_cellmask/tfrecord/RNA_tf_batch_2/tf_0.tfrecord
../concerto/online_cellmask/tfrecord/Protein_tf_batch_2/tf_0.tfrecord
all_samples num:32
all_samples num:64
all_samples num:96
all_samples num:128
all_samples num:160
all_samples num:192
all_samples num:224
all_samples num:256
all_samples num:288
all_samples num:320
all_samples num:352
all_samples num:384
all_samples num:416
all_samples num:448
all_samples num:480
all_samples num:512
all_samples num:544
all_samples num:576
all_samples num:608
all_samples num:640
all_samples num:672
all_samples num:704
all_samples num:736
all

...

Processed dataset shape: (5265, 3617)
Processed dataset shape: (5265, 224)
AnnData object with n_obs × n_vars = 5265 × 3617
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch', 'n_genes'
    uns: 'log1p'
counter: 1000 shape: (3617,), batch: 0
counter: 2000 shape: (3617,), batch: 0
counter: 3000 shape: (3617,), batch: 0
counter: 4000 shape: (3617,), batch: 0
counter: 5000 shape: (3617,), batch: 0
AnnData object with n_obs × n_vars = 5265 × 224
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch', 'n_genes'
    uns: 'log1p'
counter: 1000 shape: (224,), batch: 0
counter: 2000 shape: (224,), batch: 0
counter: 3000 shape: (224,), batch: 0
counter: 4000 shape: (224,), batch: 0
counter: 5000 shape: (224,), batch: 0
../concerto/online_cellmask/tfrecord/RNA_tf_batch_6/tf_0.tfrecord
../concerto/online_cellmask/tfrecord/Protein_tf_batch_6/tf_0.tfrecord
all_samples num:32
all_samples num:64
all_samples num:96


/root/anaconda3/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


Processed dataset shape: (8770, 3617)
Processed dataset shape: (8770, 224)
AnnData object with n_obs × n_vars = 8770 × 3617
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch', 'n_genes'
    uns: 'log1p'
counter: 1000 shape: (3617,), batch: 0
counter: 2000 shape: (3617,), batch: 0
counter: 3000 shape: (3617,), batch: 0
counter: 4000 shape: (3617,), batch: 0
counter: 5000 shape: (3617,), batch: 0
counter: 6000 shape: (3617,), batch: 0
counter: 7000 shape: (3617,), batch: 0
counter: 8000 shape: (3617,), batch: 0
AnnData object with n_obs × n_vars = 8770 × 224
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch', 'n_genes'
    uns: 'log1p'
counter: 1000 shape: (224,), batch: 0
counter: 2000 shape: (224,), batch: 0
counter: 3000 shape: (224,), batch: 0
counter: 4000 shape: (224,), batch: 0
counter: 5000 shape: (224,), batch: 0
counter: 6000 shape: (224,), batch: 0
counter: 7000 shape: (224,), batch: 0


/root/anaconda3/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:140: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number


Processed dataset shape: (8578, 3617)
Processed dataset shape: (8578, 224)
AnnData object with n_obs × n_vars = 8578 × 3617
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch', 'n_genes'
    uns: 'log1p'
counter: 1000 shape: (3617,), batch: 0
counter: 2000 shape: (3617,), batch: 0
counter: 3000 shape: (3617,), batch: 0
counter: 4000 shape: (3617,), batch: 0
counter: 5000 shape: (3617,), batch: 0
counter: 6000 shape: (3617,), batch: 0
counter: 7000 shape: (3617,), batch: 0
counter: 8000 shape: (3617,), batch: 0
AnnData object with n_obs × n_vars = 8578 × 224
    obs: 'orig.ident', 'nCount_rna', 'nFeature_rna', 'nCount_adt', 'nFeature_adt', 'l1', 'batch', 'n_genes'
    uns: 'log1p'
counter: 1000 shape: (224,), batch: 0
counter: 2000 shape: (224,), batch: 0
counter: 3000 shape: (224,), batch: 0
counter: 4000 shape: (224,), batch: 0
counter: 5000 shape: (224,), batch: 0
counter: 6000 shape: (224,), batch: 0
counter: 7000 shape: (224,), batch: 0


In [8]:
adata_all = sc.concat(adata)

### save embedding

In [9]:
for i in pd.unique(adata_all.obs['batch']):
    if not os.path.exists(pj(result_path, 'default/predict','subset_%d/z/joint'%(i))):
        os.makedirs(pj(result_path, 'default/predict','subset_%d/z/joint'%(i)))
    pd.DataFrame(adata_all[adata_all.obs['batch']==i].obsm['X_embedding']).to_csv(pj(result_path, 'default/predict','subset_%d/z/joint'%(i),'00.csv'), index=False, header=False)